<a href="https://colab.research.google.com/github/jainmadhur07/llamaindex-rag-app/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-google-genai langchain-community langchain-core

In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['GOOGLE_API_KEY'] = ''

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 16.5 MB/s eta 0:00:00


In [9]:
from google.colab import files

uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")

# Load the PDF
print("Loading PDF...")
loader1 = PyPDFLoader(filename) pages = loader1.load()
print(f"Loaded {len(pages)} pages from PDF")

Saving NCERT-Class-12-Physics-Part-1.pdf to NCERT-Class-12-Physics-Part-1 (1).pdf
Uploaded file: NCERT-Class-12-Physics-Part-1 (1).pdf
Loading PDF...
Loaded 291 pages from PDF


In [10]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 11.0 MB/s eta 0

In [11]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=800, chunk_overlap=50)
splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

In [12]:
def ask_question(question):
    # Retrieve relevant documents
    docs = retriever.invoke(question)

    # Create context from retrieved documents
    context = "\n\n".join([doc.page_content for doc in docs])

    # Simple prompt template
    prompt = f"""
    Based on the following context from the physics textbook, answer the question clearly and accurately.

    Context:
    {context}

    Question: {question}

    Answer:
    """

    # Get response from LLM
    response = llm.invoke(prompt)
    return response.content

In [13]:
def main():
    print("PDF Q&A System Ready!")
    print("Ask questions about your physics textbook. Type 'quit' to exit.\n")

    while True:
        question = input("Your question: ")

        if question.lower() == 'quit':
            print("Goodbye!")
            break

        try:
            answer = ask_question(question)
            print(f"\nAnswer: {answer}\n")
            print("-" * 50)
        except Exception as e:
            print(f"Error: {e}")

In [ ]:
if __name__ == "__main__":
    main()

PDF Q&A System Ready!
Ask questions about your physics textbook. Type 'quit' to exit.

